Analysing the data gathered with my WebScrapper
===============================================

For this notebook, I followed GitHub examples for the organization and format.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.close("all")
import numpy as np
import os 

Loading our csv file in a pandas data frame.

In [ ]:
catalogFilePath = os.path.join("..","data", "catalog.csv")
df = pd.read_csv(catalogFilePath)

#### Metrics and Analysis


  #### -- Average price:
  
  #####    Rent, condo fee and total per neigborhood

In [ ]:
colsToSum = ["rent", "condo_fee", "total_monthly_value"]
nhAvgPricesResults = df.groupby("neighborhood")[colsToSum].mean()
display(nhAvgPricesResults.round(1))

nhAvgPricesResults.round(1).plot(kind="bar", figsize=(20,6))
plt.ylabel("Average Price")
plt.show()


 #####    Price per m² per neigborhood

In [ ]:
df["pricePerM²"] = df["total_monthly_value"]/df["property_size"]
colsToSum = ["pricePerM²"]
nhresultsPricepPerM2= df.groupby("neighborhood")[colsToSum].mean()
display(nhresultsPricepPerM2.round(decimals=0))

nhresultsPricepPerM2.round(0).boxplot(column="pricePerM²", by="neighborhood", figsize=(12,6))
plt.xticks(rotation=90)
plt.ylabel("Average price")
plt.show()

  #### -- Size distribuition comparison per neighborhood

In [ ]:
colsToSum = ["property_size",  "bedrooms", "bathrooms", "parking_spaces"]
nhresultsSize = df.groupby("neighborhood")[colsToSum].mean()
display(nhresultsSize.round(decimals=0))

 ##### How the number of [bedrooms, bathrooms, parking_spaces] affects the rental price

 The comparison uses the average quantity of rentals with that especific combination of [bedrooms, bathrooms, parking_spaces] in other to be more precise about how consistent are the values(The larger the DataFrame, the more precise the values), also, just to be more pratical about the 

Below are two implementations I encountered: the first one before i research on possible approaches to implement the comparison, and the second one after.



In [ ]:
combinations = {
    "bedrooms": [],
    "bathrooms": [],
    "parking_spaces": [],
    "total_monthly_value": [],
    "count": []
}
for i in range(5):
    for j in range(5):
        for k in range(5):
            cc = (
                df[(df["bedrooms"] == i) & 
                (df["bathrooms"] == j) & 
                (df["parking_spaces"] == k)]
            )
            count = len(cc)
            if not cc.empty:       
                newcc = cc["total_monthly_value"].mean().round(1).item()
                combinations["bedrooms"].append(i)
                combinations["bathrooms"].append(j)
                combinations["parking_spaces"].append(k)
                combinations["total_monthly_value"].append(newcc)
                combinations["count"].append(count)

sizeCompDataFrame = pd.DataFrame(data=combinations)
averageCount = sizeCompDataFrame["count"].mean().round(0).item()
filteredSizeCompDataFrame = sizeCompDataFrame[sizeCompDataFrame["count"] >= averageCount]
display(filteredSizeCompDataFrame.drop('count', axis=1))



In [ ]:
groupbyAvarege_prices = df.groupby(["bedrooms", "bathrooms", "parking_spaces"])["total_monthly_value"].mean().reset_index(name="mean_price")
counts = df.groupby(["bedrooms", "bathrooms", "parking_spaces"]).size().reset_index(name="count")

combinations = counts.merge(groupbyAvarege_prices, on=["bedrooms", "bathrooms", "parking_spaces"]) #Merging the two lists, to get the total month value and the count of the number of rentals that satisfies one combination.

averageCount = combinations["count"].median() #The average count to have a better consistency of values after use .mean().
filteredCombinations = combinations[combinations["count"] >= averageCount]
display(filteredCombinations.drop('count', axis=1).style.hide(axis="index"))

groups = filteredCombinations.groupby(['bedrooms', 'bathrooms'])['mean_price'].mean().unstack()


## https://www.youtube.com/watch?v=0U9cs2V-Mqc&pp=ygUWaGVhdG1hcCBzZWFib3JuIHB5dGhvbg%3D%3D, tutorial that i used and had the inspiration to do a heatmap.
heatmap_data = filteredCombinations.pivot_table(index='bathrooms', columns='bedrooms', values='mean_price')

plt.figure(figsize=(8,6))
sns.heatmap(heatmap_data, annot=True, fmt=".0f", cmap='YlGnBu')
plt.title('Heatmap of Rental Price')
plt.xlabel('Bedrooms')
plt.ylabel('Bathrooms')
plt.show()